In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, concatenate, Dropout


In [38]:
books_df = pd.read_csv("Books.csv")

<ipython-input-38-ddc662c0dc8f>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv("Books.csv")


In [39]:
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [40]:
users_df = pd.read_csv("Users.csv")
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [41]:
ratings_df = pd.read_csv("Ratings.csv")
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
len(ratings_df)

1149780

In [ ]:
ratings_df.loc[(ratings_df['Book-Rating'] == 0)]

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
2,276727,0446520802,0
5,276733,2080674722,0
10,276746,0425115801,0
11,276746,0449006522,0
...,...,...,...
1149769,276704,059032120X,0
1149770,276704,0679752714,0
1149772,276704,080410526X,0
1149774,276704,0876044011,0


In [42]:
books_rating_df = books_df.merge(ratings_df,on='ISBN')
books_rating_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [ ]:
len(books_rating_df)

1031136

In [43]:
books_rating_user_df = books_rating_df.merge(users_df,on='User-ID')
books_rating_user_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating,Location,Age
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0,"stockton, california, usa",18.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5,"timmins, ontario, canada",NaN
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,8,0,"timmins, ontario, canada",NaN
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,8,0,"timmins, ontario, canada",NaN
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,8,0,"timmins, ontario, canada",NaN


In [ ]:
len(books_rating_user_df)

1031136

In [44]:
books_rating_user_df['Age'].fillna(books_rating_user_df['Age'].mode().iloc[0], inplace = True)

In [45]:
books_rating_user_df = books_rating_user_df.drop(columns=['Location','Image-URL-S','Image-URL-M','Image-URL-L'])
books_rating_user_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Age
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0,18.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,5,33.0
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,8,0,33.0
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,8,0,33.0
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,8,0,33.0


In [ ]:
# books_rating_user_df.drop(books_rating_user_df[books_rating_user_df['Book-Rating'] == 0].index, inplace=True)

In [ ]:
books_rating_user_df.nunique()

ISBN           270151
Book-Title     241071
Book-Author    101587
User-ID         92106
Book-Rating        11
dtype: int64

In [ ]:
len(books_rating_user_df)

383842

In [ ]:
books_rating_user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031136 entries, 0 to 1031135
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   ISBN         1031136 non-null  int64 
 1   Book-Title   1031136 non-null  int64 
 2   Book-Author  1031134 non-null  object
 3   User-ID      1031136 non-null  int64 
 4   Book-Rating  1031136 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 39.3+ MB


In [46]:
#Convert non numeric features to numeric values using LabelEncoder
label_encoder_isbn = LabelEncoder()
books_rating_user_df['ISBN'] = label_encoder_isbn.fit_transform(books_rating_user_df['ISBN'])

In [47]:
books_rating_user_df['Book-Title'] = label_encoder_isbn.fit_transform(books_rating_user_df['Book-Title'])
# books_rating_user_df['Book-Author'] = label_encoder_isbn.fit_transform(books_rating_user_df['Book-Author'])


In [ ]:
label_encoder_isbn.transform(['The Two Towers'])

array([211379])

In [16]:
books_rating_user_df.head()

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
0,24927,36197,Mark P. O. Morford,2,0
1,73,36036,Richard Bruce Wright,8,5
2,8175,47448,Carlo D'Este,8,0
3,59963,68219,Gina Bari Kolata,8,0
4,71447,198739,E. J. W. Barber,8,0


In [ ]:
len(books_rating_user_df)

1031136

In [63]:
books_rating_user_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Age
0,24927,36197,Mark P. O. Morford,2002,Oxford University Press,2,0,18.0
1,73,36036,Richard Bruce Wright,2001,HarperFlamingo Canada,8,5,33.0
2,8175,47448,Carlo D'Este,1991,HarperPerennial,8,0,33.0
3,59963,68219,Gina Bari Kolata,1999,Farrar Straus Giroux,8,0,33.0
4,71447,198739,E. J. W. Barber,1999,W. W. Norton &amp; Company,8,0,33.0


In [51]:
train, test = train_test_split(books_rating_user_df, test_size = 0.2, random_state = 0)

In [55]:
# books_rating_user_df.loc[books_rating_user_df['User-ID'].idxmax()]
books_rating_user_df['User-ID'].max()

278854

In [59]:
books_rating_user_df['Book-Title'].max()

241070

In [88]:
n_books = books_rating_user_df['Book-Title'].max()
n_users = books_rating_user_df['User-ID'].max()


In [89]:
user_input = Input(shape=(1,))
book_input = Input(shape=(1,))
# Embedding - Turns positive integers (indexes) into dense vectors of fixed size
user_embed = Embedding(n_users+1, 50)(user_input)
book_embed = Embedding(n_books+1, 50)(book_input)
# padded_inputs = tf.keras.utils.pad_sequences(user_embed, padding="post")
#Convert parameters to one dimension
user_flat = Flatten()(user_embed)
book_flat = Flatten()(book_embed)
concat = concatenate([book_flat,user_flat])
dense1 = Dense(128, activation='relu')(concat)
#The Dropout layer randomly sets input units to 0 with a frequency of rate at each step during training time, which helps prevent overfitting.
dropout1 = Dropout(0.1)(dense1)
dense2 = Dense(64, activation='relu')(dropout1)
# dense2 = Dense(64, activation='relu')(dense1)
dropout2 = Dropout(0.1)(dense2)
dense3 = Dense(32, activation='relu')(dropout2)
# dense3 = Dense(32, activation='relu')(dense2)
output = Dense(1)(dense3)

model = Model(inputs = [book_input,user_input], outputs = output)
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics=["mse", "mae"])

model.summary()

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_40 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 input_39 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 embedding_37 (Embedding)    (None, 1, 50)                1205355   ['input_40[0][0]']            
                                                          0                                       
                                                                                                  
 embedding_36 (Embedding)    (None, 1, 50)                1394275   ['input_39[0][0]']     

In [90]:
model.fit([train['Book-Title'], train['User-ID']], train['Book-Rating'], epochs = 3, batch_size = 512, verbose = 1)

Epoch 1/3
1612/1612 [==============================] - 1031s 638ms/step - loss: 11.9030 - mse: 11.9030 - mae: 2.8084
Epoch 2/3
1612/1612 [==============================] - 1022s 634ms/step - loss: 9.6042 - mse: 9.6042 - mae: 2.3601
Epoch 3/3
1612/1612 [==============================] - 1034s 642ms/step - loss: 7.5643 - mse: 7.5643 - mae: 1.9115


In [92]:
model_loss = model.evaluate([test['Book-Title'], test['User-ID']], test['Book-Rating'])
print(f"Loss: {model_loss}")

6445/6445 [==============================] - 11s 2ms/step - loss: 12.5936 - mse: 12.5936 - mae: 2.7228
Loss: [12.59360122680664, 12.59360122680664, 2.7228081226348877]


In [104]:
model.save("recommendation.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [93]:
books_rating_user_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Age
0,24927,36197,Mark P. O. Morford,2002,Oxford University Press,2,0,18.0
1,73,36036,Richard Bruce Wright,2001,HarperFlamingo Canada,8,5,33.0
2,8175,47448,Carlo D'Este,1991,HarperPerennial,8,0,33.0
3,59963,68219,Gina Bari Kolata,1999,Farrar Straus Giroux,8,0,33.0
4,71447,198739,E. J. W. Barber,1999,W. W. Norton &amp; Company,8,0,33.0


In [99]:
def recommend(book_title, model, books_rating_user_df, n_recommend=5):
  # label_encoder_isbn.transform(['Classical Mythology'])
  book_encoded = label_encoder_isbn.transform([book_title])
  # print(book_encoded)
  books = np.array([book_encoded] * len(books_rating_user_df['User-ID']))
  ratings = model.predict([books, books_rating_user_df['User-ID']])

  df_new = books_rating_user_df.assign(Ratings=ratings)
  df_new = df_new.sort_values(by='Ratings',ascending = False)
  df_new = df_new.drop_duplicates(subset=['Book-Title'])
  df_new['Book-Title'] = label_encoder_isbn.inverse_transform(df_new['Book-Title'])

  return df_new[['Book-Title']]

In [ ]:
len(books_rating_user_df['ISBN'])

1031136

In [106]:
book = 'The Lord of the Rings'
top_recommend = recommend(book, model, books_rating_user_df)
print("Top 5 recommendations for book",book)
top_recommend.head(10)

32223/32223 [==============================] - 51s 2ms/step
Top 5 recommendations for book The Lord of the Rings


,Book-Title
764766,The Green Mile: The Complete Serial Novel
764767,"Assassins: Assignment: Jerusalem, Target: Anti..."
764781,Scary Stories 3 : More Tales to Chill Your Bon...
764774,Apollyon: The Destroyer Is Unleashed (Left Beh...
764762,Left Behind: A Novel of the Earth's Last Days ...
764763,The Book of Lost Tales 2 (The History of Middl...
764764,The Book of Lost Tales 1 (The History of Middl...
764765,The Hobbit : The Enchanting Prelude to The Lor...
764768,More Scary Stories To Tell In The Dark
764769,Insomnia
